## Scheduling

    所有像Dask Array，Dask dataframe和Dask bag这样的大规模Dask集合以及delayed, futures这些API都会生成任务图，其中图中的每个节点都是普通的Python函数，节点之间的边是普通的Python由一个任务创建的对象作为ouptuts并用作另一个任务的输入。 Dask生成这些任务图后，需要在并行硬件上执行它们。这是任务调度器的工作。存在不同的任务调度程序。每个将消耗一个任务图并计算相同的结果，但具有不同的性能特征。
![task_graph](collections-schedulers.png)
***
**Dask的调度程序：**
+ Single machine scheduler:    
    1. synchronous
    2. multiprocessing 
    3. threaded     
+ Distributed scheduler

### 单机任务调度

**synchronous**: 适用于程序调试过程(默认)     
**multiprocessing**: 将数据移动到远程进程并返回会导致性能损失，特别是在进程之间传输的数据很大时。当工作流程相对线性时，多处理调度程序是一个很好的选择。     
**thread**: 因为*GIL(全局解释锁)*的存在，有些计算密集的程序无法实现并行
***
#### 快速上手

In [1]:
%%time
from dask.delayed import delayed
import pandas as pd 
import dask
dask.config.set(scheduler='synchronous')

files = ['data1/1.csv',
         'data1/2.csv',
         'data1/3.csv',
         'data1/4.csv']
tasks = []

for file in files:
    df = delayed(pd.read_csv)(file)
    tasks.append(df['Key'].count())

result = delayed(sum)(tasks)
result.compute()

CPU times: user 11.2 s, sys: 2.18 s, total: 13.4 s
Wall time: 14.7 s


In [2]:
%%time
dask.config.set(scheduler='processes')  # overwrite default with multiprocessing scheduler
result.compute()

CPU times: user 31 ms, sys: 51.3 ms, total: 82.3 ms
Wall time: 8.02 s


In [3]:
%%time
dask.config.set(scheduler='threads')   # overwrite default with threadingpool scheduler
result.compute()

CPU times: user 16.7 s, sys: 3.55 s, total: 20.3 s
Wall time: 7.32 s


### 分布式任务调度
#### 快速上手

In [ ]:
from dask.distributed import Client
client = Client()
client

**Dashboard简介**

In [ ]:
import dask.array as da
x = da.random.random((10000, 10000, 10), chunks=(1000, 1000, 5))
y = da.random.random((10000, 10000, 10), chunks=(1000, 1000, 5))
z = (da.arcsin(x) + da.arccos(y)).sum(axis=(1,2))
z.compute()

**在集群上启动运算**     
通过map/submit方法在集群上启动运算

In [ ]:
def square(x):
    return x * x

A = client.map(square, range(1, 10))
total = client.submit(sum, A)
total

In [ ]:
total.result()

**访问future对象的运算结果**

In [ ]:
client.gather(A)
#client.gather(total)

**节点之间传递数据集**


In [ ]:
import pandas as pd

df = pd.read_csv('data1/1.csv').head()
client.publish_dataset(df1=df)   # save value

In [ ]:
client.list_datasets()            #获取dataset列表
df = client.get_dataset('df1')    #获取dataset
df

### Streaming
dask推荐的流式处理方式是使用python标准库的Queue。


